# Imports

In [17]:
#all imports
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from IPython.display import display
from nltk import TweetTokenizer
import pickle
import os

# label conversion dictionaries: text to num, num to text

In [18]:
dic_aggression_level = {
    'NAG' : 1,
    'CAG' : 2,
    'OAG' : 3
}

dic_reverse_aggression_level = {}
for i in dic_aggression_level:
    dic_reverse_aggression_level[dic_aggression_level[i]] = i
    
print(dic_aggression_level, '\n', dic_reverse_aggression_level)

{'OAG': 3, 'CAG': 2, 'NAG': 1} 
 {1: 'NAG', 2: 'CAG', 3: 'OAG'}


PREPARING DATA WITH PANDAS
----

In [25]:
#labels
dic_aggression_level = {
    'NAG' : 1,
    'CAG' : 2,
    'OAG' : 3
}

dic_reverse_aggression_level = {}
for i in dic_aggression_level:
    dic_reverse_aggression_level[dic_aggression_level[i]] = i
    
print(dic_aggression_level, '\n', dic_reverse_aggression_level)

#train data
train_pd = shuffle(pd.concat((pd.read_csv("train.csv")[['Data', 'Label']], pd.read_csv("valid.csv")[['Data', 'Label']])), random_state=20)
# train_pd['Label'].replace('CAG', 'OAG', inplace=True)
train_pd['Label_num'] = train_pd.Label.map(dic_aggression_level)

#test data
# test_fb_pd = shuffle(pd.read_csv("test_fb.csv")[['Data', 'Label']], random_state=20)
# test_fb_pd['Label_num'] = test_fb_pd.Label.map(dic_aggression_level)
# test_tw_pd = shuffle(pd.read_csv("test_tw.csv")[['Data', 'Label']], random_state=20)
# test_tw_pd['Label_num'] = test_tw_pd.Label.map(dic_aggression_level)

#test data
test_pd = pd.read_csv("test_fb.csv")
test_pd.drop('ID',1,inplace=True)
test_pd = shuffle(test_pd, random_state = 20)

# merge binary classification (CAG -> OAG)
# test_pd['Label'].replace('CAG', 'OAG', inplace=True)

test_pd['Label_num'] = test_pd.Label.map(dic_aggression_level)

print("\n\n\Train DATA")
print(train_pd.Label.value_counts())
display(train_pd.head(10))


print("\n\n\nTEST DATA")
print(test_pd.Label.value_counts())
display(test_pd.head(10))

#individual classes
class_wise_train_data = {}

class_wise_test_data = {}

for i in range(1,4):
    class_wise_train_data[i] = train_pd[(train_pd['Label_num']==i)]
    class_wise_test_data[i] = test_pd[(test_pd['Label_num']==i)]
    print(class_wise_train_data[i].Label.value_counts())
    display(class_wise_train_data[i].head(10))
    print(class_wise_test_data[i].Label.value_counts())
    display(class_wise_test_data[i].head(10))


{'OAG': 3, 'CAG': 2, 'NAG': 1} 
 {1: 'NAG', 2: 'CAG', 3: 'OAG'}


\Train DATA
OAG    8716
NAG    6285
Name: Label, dtype: int64


,Data,Label,Label_num
2930,Focus on making cash available then only peo...,OAG,3
5103,She's so ignorant Megha Mukherji,OAG,3
5090,"Sonia I am holding Rel cap 430, please suggest...",NAG,1
9074,why dont u make ur room sound proof..simple,OAG,3
6769,Showing everything and saying bold...,OAG,3
140,Then what happens in pantry coach dedicated fo...,OAG,3
1756,We should respect every religion. May be he wa...,OAG,3
7272,Car is good.. bt i must say.. i only heard 'aa...,NAG,1
10305,friends we have to understand the ground reali...,OAG,3
45,Bad...........,OAG,3





TEST DATA
NAG    630
OAG    286
Name: Label, dtype: int64


,Data,Label,Label_num
156,What will be done to the money we have ?,NAG,1
211,Unchange the rapo rate could lead the stagnate...,NAG,1
13,PK Movie Bhagawan Shiv Ko Aapman Kiya.. I Ha...,NAG,1
798,Thousands of people have died due to bandhs an...,OAG,3
640,Worst F.M ever.what about 5 lakhs tax limit? ...,NAG,1
568,Pak army rape modi daughter and go back. Now m...,NAG,1
321,I am clean without cash as transactions are th...,NAG,1
119,I have 5000 shares of Pnb @75.90 please tell m...,NAG,1
820,these bhagwa terrorists can't digest their mea...,OAG,3
721,Worst Decision and very worst implementation. ...,OAG,3


NAG    6285
Name: Label, dtype: int64


...

,Data,Label,Label_num


OAG    8716
Name: Label, dtype: int64


,Data,Label,Label_num
2930,Focus on making cash available then only peo...,OAG,3
5103,She's so ignorant Megha Mukherji,OAG,3
9074,why dont u make ur room sound proof..simple,OAG,3
6769,Showing everything and saying bold...,OAG,3
140,Then what happens in pantry coach dedicated fo...,OAG,3
1756,We should respect every religion. May be he wa...,OAG,3
10305,friends we have to understand the ground reali...,OAG,3
45,Bad...........,OAG,3
3168,It's like a devil think .. what tha hell is th...,OAG,3
5374,Third rate dog,OAG,3


OAG    286
Name: Label, dtype: int64


,Data,Label,Label_num
798,Thousands of people have died due to bandhs an...,OAG,3
820,these bhagwa terrorists can't digest their mea...,OAG,3
721,Worst Decision and very worst implementation. ...,OAG,3
797,Wow miracle of modi Manmohan started speaking,OAG,3
253,Yes yes ..traffic population pollution unlivab...,OAG,3
819,Pakistanis are not human being. They are just ...,OAG,3
707,F. PM Manmohan Singh had 10 years now he has n...,OAG,3
684,"Hello karki , How are you?A troll paid by BJP....",OAG,3
755,"The most useless PM India ever seen, he could ...",OAG,3
906,When asked to speak in Parliament ran away. Sp...,OAG,3


# Brown Clustering

In [ ]:
brown_clustering_dict = pickle.load(open("brown_clustering_dictionary.pickle", 'rb'))
brown_clustering_keys_set = set(brown_clustering_dict.keys())
from nltk import TweetTokenizer
tweeter = TweetTokenizer()

def get_brown_cluster(word):
    if word in brown_clustering_keys_set:
        return brown_clustering_dict[word]
    return ''

train_pd['Data'] = pd.Series(map(" ".join, list(map(lambda x: tuple(map(get_brown_cluster, x)), tuple(map(tweeter.tokenize, train_pd['Data']))))))
test_pd['Data'] = pd.Series(map(" ".join, list(map(lambda x: tuple(map(get_brown_cluster, x)), tuple(map(tweeter.tokenize, test_pd['Data']))))))

#over/under sampling
train_pd = train_pd[train_pd['Label'] == 'OAG'][:3000].append(
            train_pd[train_pd['Label'] == 'CAG'][:3000].append(
            train_pd[train_pd['Label'] == 'NAG'][:3000]))

print(train_pd['Data'], test_pd['Data'])

# POS Tagging

In [8]:
%%time
from multiprocessing import Pool

#tokenize
from nltk.tokenize import TweetTokenizer
tweeter = TweetTokenizer()
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(tweeter.tokenize, train_pd['Data']))
pos_test  = list(process_pool.map(tweeter.tokenize, test_pd['Data']))
process_pool.close()
print("Finished tokenising")

#remove stop words
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
def f(x):
    return tuple(filter(lambda y: y not in stop_words, x))
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_test = list(process_pool.map(f , pos_test))
process_pool.close()
print("Finished removing stop words")

#stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def f(x):
    return tuple(map(ps.stem, x))
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_test = list(process_pool.map(f , pos_test))
process_pool.close()
print("Finished stemming")

#POS tagging
from nltk import pos_tag
process_pool = Pool(processes=55)
pos_train = list(process_pool.map(pos_tag, pos_train))
pos_test = list(process_pool.map(pos_tag, pos_test))
process_pool.close()
print("Finished POS tagging")

#recombination
# def f(x): #pos concatenation
#     return " ".join(list(map("".join, x)))

def f(x): #pos only sentences
    return " ".join(list(map(lambda y: y[1], x)))

process_pool = Pool(processes=55)
pos_train = list(process_pool.map(f, pos_train))
pos_test = list(process_pool.map(f, pos_test))
process_pool.close()
print("Finished recombining")


print(pos_train[:10], pos_test[:10])
print(len(pos_train))

Finished tokenising
Finished removing stop words
Finished stemming
Finished POS tagging
Finished recombining
['NNS VBP NN NN NN NN . CC JJ JJ NN VBD JJ NN NN . NN NN', 'NNS VBP NN NN', 'NN PRP VBP JJ NN CD , NNS VBP', 'NN JJ VBP JJ NN NN NN NNP NN', 'NN NNS VBP NN :', 'RB VB RB NN JJ NN NN NN NNP NN NN NN , NN NNS VBP JJ NN . IN JJ NNS VBP PRP VB NN NN : NN NN NN VBP VBN NN NN JJ NN :', 'PRP VBP DT NN . MD VB JJ CC VB DT NN .', "NN JJ NN NN MD VB NNP NN '' JJ NNP NN NNP NN NNP VBZ NNP POS NN NN NN", 'NN NN NN NN , JJ JJ NN CD NN NN , NN RB JJ NN .', 'JJ :'] ['WP VBN NN .', 'JJ NN NN MD VB JJ NN NN NN NN NN .', 'NNP NN NN NN NNP NN VBD NNP PRP VBP JJ NN : PRP VBP JJ NNP NN NNP NN NNP NN NNP NNP NNP VBD NN : NN NN NN JJ NN NN RBR JJ JJ NN NN NN NN NNP CC VB', 'CD NN NN JJ NN NN . WP JJ NN .', 'RB NNP . NNP RB CD JJ NN NN . , VBD NN .', 'NN NN NN NN NN VBP RB . RB JJ VBP CD JJ NN NN', 'PRP VBP IN NN NN IN NN PRP VBP', 'PRP CD NN JJ NNP . CD NNS VBP VB CD VB', 'JJ JJ NN JJS NN IN JJ NN NN 

# Declaring different types of features (in terms if TFIDvectorizer)

In [26]:
# TfidfVectorizer(input=’content’, encoding=’utf-8’, decode_error=’strict’, strip_accents=None, lowercase=True, 
#                 preprocessor=None, tokenizer=None, analyzer=’word’, stop_words=None, 
#                 token_pattern=’(?u)\b\w\w+\b’, ngram_range=(1, 1), max_df=1.0, min_df=1, 
#                 max_features=None, vocabulary=None, binary=False, dtype=<class ‘numpy.int64’>, 
#                 norm=’l2’, use_idf=True, smooth_idf=True, sublinear_tf=False)

max_features = 20000

feature_dict = {
    'unigram' : TfidfVectorizer(max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    'bigram'  : TfidfVectorizer(ngram_range=(2,2), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    'trigram'  : TfidfVectorizer(ngram_range=(3,3), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    'quadgram'  : TfidfVectorizer(ngram_range=(4,4), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    'fivegram'  : TfidfVectorizer(ngram_range=(5,5), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    '1-2_gram'  : TfidfVectorizer(ngram_range=(1, 2), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    '1-3_gram'  : TfidfVectorizer(ngram_range=(1, 3), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    '1-4_gram'  : TfidfVectorizer(ngram_range=(1, 4), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    '2-3_gram'  : TfidfVectorizer(ngram_range=(2, 3), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    '2-4_gram'  : TfidfVectorizer(ngram_range=(2, 4), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    '3-4_gram'  : TfidfVectorizer(ngram_range=(3, 4), max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     'unigram_without_stopwords' : TfidfVectorizer(stop_words='english', 
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     'bigram_without_stopwords'  : TfidfVectorizer(ngram_range=(2,2), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     'trigram_without_stopwords'  : TfidfVectorizer(ngram_range=(3,3), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     'quadgram_without_stopwords'  : TfidfVectorizer(ngram_range=(4,4), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     'fivegram_without_stopwords'  : TfidfVectorizer(ngram_range=(5,5), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     '1-2_gram_without_stopwords'  : TfidfVectorizer(ngram_range=(1, 2), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     '1-3_gram_without_stopwords'  : TfidfVectorizer(ngram_range=(1, 3), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     '1-4_gram_without_stopwords'  : TfidfVectorizer(ngram_range=(1, 4), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     '2-3_gram_without_stopwords'  : TfidfVectorizer(ngram_range=(2, 3), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     '2-4_gram_without_stopwords'  : TfidfVectorizer(ngram_range=(2, 4), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
#     '3-4_gram_without_stopwords'  : TfidfVectorizer(ngram_range=(3, 4), stop_words='english',
#                                                   max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
    '1-1_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,1), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'1-2_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,2), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'1-3_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,3), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'1-4_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,4), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'1-5_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,5), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'1-6_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,6), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'1-7_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,7), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'1-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(1,8), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'2-2_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(2,2), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'2-3_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(2,3), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'2-4_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(2,4), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'2-5_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(2,5), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'2-6_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(2,6), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'2-7_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(2,7), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'2-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(2,8), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'3-3_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(3,3), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'3-4_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(3,4), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'3-5_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(3,5), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'3-6_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(3,6), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'3-7_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(3,7), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
	'3-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(3,8), 
                                    max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'4-4_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(4,4), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'4-5_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(4,5), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'4-6_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(4,6), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'4-7_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(4,7), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'4-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(4,8), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'5-5_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(5,5), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'5-6_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(5,6), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'5-7_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(5,7), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'5-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(5,8), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'6-6_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(6,6), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'6-7_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(6,7), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'6-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(6,8), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'7-7_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(7,7), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'7-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(7,8), 
#                                     max_features=max_features, min_df=3, tokenizer=TweetTokenizer().tokenize),
# 	'8-8_char_gram'  : TfidfVectorizer(analyzer='char', ngram_range=(8,8), 
#                                     max_features=max_features, min_df=3)
}

# Extra features (emoji, punctuation, capitals)

In [10]:
extra_feature_dict = {
    'char_1_gram_non_alphabet' : TfidfVectorizer(ngram_range=(1, 1), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'char_2_gram_non_alphabet' : TfidfVectorizer(ngram_range=(2, 2), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'char_3_gram_non_alphabet' : TfidfVectorizer(ngram_range=(3, 3), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'char_4_gram_non_alphabet' : TfidfVectorizer(ngram_range=(4, 4), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'char_5_gram_non_alphabet' : TfidfVectorizer(ngram_range=(5, 5), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'char_6_gram_non_alphabet' : TfidfVectorizer(ngram_range=(6, 6), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'char_7_gram_non_alphabet' : TfidfVectorizer(ngram_range=(7, 7), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'char_8_gram_non_alphabet' : TfidfVectorizer(ngram_range=(8, 8), max_features=max_features, 
                                                 token_pattern='[^\w+\s+]'),
    'capitals_char_1_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z]'),
    'capitals_char_2_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z][A-Z]'),
    'capitals_char_3_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z][A-Z][A-Z]'),
    'capitals_char_4_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z][A-Z][A-Z][A-Z]'),
    'capitals_char_5_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z][A-Z][A-Z][A-Z][A-Z]'),
    'capitals_char_6_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z][A-Z][A-Z][A-Z][A-Z][A-Z]'),
    'capitals_char_7_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z][A-Z][A-Z][A-Z][A-Z][A-Z][A-Z]'),
    'capitals_char_8_gram' : TfidfVectorizer(max_features=max_features, lowercase=False,
                        token_pattern='[A-Z][A-Z][A-Z][A-Z][A-Z][A-Z][A-Z][A-Z]'),
}

# POS Features

In [6]:
pos_feature_dict = {
    'unigram_pos' : TfidfVectorizer(max_features=max_features, min_df=3),
    'bigram_pos'  : TfidfVectorizer(ngram_range=(2,2), max_features=max_features, min_df=3),
    'trigram_pos'  : TfidfVectorizer(ngram_range=(3,3), max_features=max_features, min_df=3),
    'quadgram_pos'  : TfidfVectorizer(ngram_range=(4,4), max_features=max_features, min_df=3),
    'fivegram_pos'  : TfidfVectorizer(ngram_range=(5,5), max_features=max_features, min_df=3),
    '1-2_gram_pos'  : TfidfVectorizer(ngram_range=(1, 2), max_features=max_features, min_df=3),
    '1-3_gram_pos'  : TfidfVectorizer(ngram_range=(1, 3), max_features=max_features, min_df=3),
    '1-4_gram_pos'  : TfidfVectorizer(ngram_range=(1, 4), max_features=max_features, min_df=3),
    '2-3_gram_pos'  : TfidfVectorizer(ngram_range=(2, 3), max_features=max_features, min_df=3),
    '2-4_gram_pos'  : TfidfVectorizer(ngram_range=(2, 4), max_features=max_features, min_df=3),
    '3-4_gram_pos'  : TfidfVectorizer(ngram_range=(3, 4), max_features=max_features, min_df=3),
}

# Pickle Everything!!!!!!!!!!!! MWUAHAHAHAHAHAHA!!!!!!!!!!!!!!!

In [23]:
train_pd['Label'].value_counts()

OAG    8716
NAG    6285
Name: Label, dtype: int64

In [27]:
%%time


count = 0
total = len(feature_dict)

# # dump textual data
pickle.dump({"Train": train_pd, "Test": test_pd}, open("original_data_in_text.pickle", 'wb'))

# # pre-empty directories
for folder in ["features/", "extra_features/", "pos_features/"]:
    remove_count = 0
    for file in os.listdir(folder):
        os.remove(os.path.join(folder, file))
        remove_count += 1
    print("Removed {} files from {}".format(remove_count, folder))

# # dump word and character n-gram tfid's
for vect in feature_dict:
    tfidf_vectorizer = feature_dict[vect]
    pickle.dump({'X_train': tfidf_vectorizer.fit_transform(train_pd.Data),
                 'y_train': train_pd.Label_num, 
                 'X_test': tfidf_vectorizer.transform(test_pd.Data), 
                 'y_test': test_pd.Label_num},
                open("features/" + vect + ".pickle", 'wb'))
    count += 1
    print("Written \t({} out of {}) \t\t {}".format(count, total, vect))
    
# dump punctuation and capital n-gram tfid's
# print("\n\n")
# count = 0
# total = len(extra_feature_dict)
# for vect in extra_feature_dict:
#     tfidf_vectorizer = extra_feature_dict[vect]
#     pickle.dump({'X_train': tfidf_vectorizer.fit_transform(train_pd.Data),
#                  'y_train': train_pd.Label_num, 
#                  'X_test': tfidf_vectorizer.transform(test_pd.Data), 
#                  'y_test': test_pd.Label_num},
#                 open("extra_features/" + vect + ".pickle", 'wb'))
#     count += 1
#     print("Written \t({} out of {}) \t\t {}".format(count, total, vect))
    
# # dump pos n-gram tfid's
# print("\n\n")
# count = 0
# total = len(pos_feature_dict)
# for vect in pos_feature_dict:
#     tfidf_vectorizer = pos_feature_dict[vect]
#     pickle.dump({'X_train': tfidf_vectorizer.fit_transform(pos_train),
#                  'y_train': train_pd.Label_num, 
#                  'X_test': tfidf_vectorizer.transform(pos_test), 
#                  'y_test': test_pd.Label_num},
#                 open("pos_features/" + vect + ".pickle", 'wb'))
#     count += 1
#     print("Written \t({} out of {}) \t\t {}".format(count, total, vect))

Removed 58 files from features/
Removed 16 files from extra_features/
Removed 0 files from pos_features/
Written 	(1 out of 32) 		 1-1_char_gram
Written 	(2 out of 32) 		 fivegram
Written 	(3 out of 32) 		 quadgram
Written 	(4 out of 32) 		 3-3_char_gram
Written 	(5 out of 32) 		 2-4_char_gram
Written 	(6 out of 32) 		 3-7_char_gram
Written 	(7 out of 32) 		 3-4_gram
Written 	(8 out of 32) 		 3-5_char_gram
Written 	(9 out of 32) 		 3-4_char_gram
Written 	(10 out of 32) 		 1-2_gram
Written 	(11 out of 32) 		 1-2_char_gram
Written 	(12 out of 32) 		 1-7_char_gram
Written 	(13 out of 32) 		 2-2_char_gram
Written 	(14 out of 32) 		 1-3_char_gram
Written 	(15 out of 32) 		 2-8_char_gram
Written 	(16 out of 32) 		 1-8_char_gram
Written 	(17 out of 32) 		 3-8_char_gram
Written 	(18 out of 32) 		 2-5_char_gram
Written 	(19 out of 32) 		 1-4_gram
Written 	(20 out of 32) 		 unigram
Written 	(21 out of 32) 		 2-4_gram
Written 	(22 out of 32) 		 1-4_char_gram
Written 	(23 out of 32) 		 2-3_gram
Wr

count_dict = {
    'NAG' : 0,
    'CAG' : 0,
    'OAG' : 0
}

count = 10

for _, row in test_pd.iterrows():
    if row['Data'].isupper():
        print(row['Label'], "\n", row['Data'], "\n\n")